<a href="https://colab.research.google.com/github/FoxPowerGH/GenAI-Journey/blob/main/GenAI_01_03_Multi_Prompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Optimización de la Interacción con  un LLM**

**Gestión de Solicitudes Múltiples**


Tradicionalmente, la generación de código a través de modelos de lenguaje (LLM) se ha basado en el envío de un único mensaje que produce un resultado inmediato. No obstante, una estrategia más efectiva y flexible consiste en adoptar un enfoque conversacional, en el cual el usuario puede interactuar con el modelo para solicitar modificaciones y optimizar la precisión de la respuesta.

Este enfoque conversacional permite ajustar gradualmente los resultados, facilitando la producción de soluciones más alineadas con los requerimientos específicos. Sin embargo, en ciertos casos, puede ser ventajoso consolidar la interacción en un único mensaje final que sintetice la conversación previa. Este método no solo mejora la claridad y coherencia del código resultante, sino que también contribuye a su mantenibilidad, al reducir la dependencia de múltiples interacciones intermedias.

La elección entre un proceso iterativo o un mensaje unificado debe evaluarse en función de la complejidad del problema y los objetivos del desarrollo, priorizando la eficiencia y la calidad del código generado.


## **Interacción Conversacional con un LLM: Optimización y Mantenibilidad**



En este módulo, exploraremos cómo aprovechar esta interacción dinámica para solicitar modificaciones y optimizar la calidad de las respuestas producidas por el LLM. Asimismo, analizaremos la posibilidad de consolidar la conversación en un único mensaje final que sintetice el intercambio previo. Este enfoque no solo contribuye a la coherencia del resultado, sino que también facilita la mantenibilidad del código, al reducir la fragmentación del proceso de generación de respuestas.

## **Configuración del Ambiente - Google CoLab**

In [1]:
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# GoogleAI Secrets
if COLAB:
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
    os.environ["SYSTEM_INSTRUCTION"] = userdata.get('SYSTEM_INSTRUCTION')

# Instalo las librerías requeridas en CoLab
if COLAB:
    #!pip install langchain langchain_openai
    !pip install -U  langchain-google-genai
    !pip install -U -q "google-generativeai>=0.8.4"

Note: using Google CoLab
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.3 MB/s eta 0:00:00


## **Veamos el resultado de esta consulta a nuestro LLM**

In [6]:
# Definimos las librerías con las que vamos a trabajar
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.prompts.chat import PromptTemplate, ChatPromptTemplate
from langchain.schema import HumanMessage, AIMessage
from IPython.display import display_markdown

TEMPLATE = """La siguiente es una conversación amigable entre un humano y una IA para generar
código en Python. Si tienes notas sobre el código, colócalas antes del código. Cualquier nota
sobre la ejecución debe ir después del código. Si mezclas notas con el código, conviértelas
en comentarios.
Asegúrate de agregar comentarios adecuados dentro del código, ordenar los imports y seguir el formato PEP-8.

Current conversation:
{history}
Human: {input}
Code Assistant:"""

PROMPT_TEMPLATE = PromptTemplate(input_variables=["history", "input"], template=TEMPLATE)


def start_conversation():
    # Realizamos la consulta al modelo LLM con la API key
    llm =  ChatGoogleGenerativeAI(
      model="gemini-1.5-flash",
      temperature= 0.0,
      max_tokens= 1024)

    # Inicializamos memory and conversation
    memory = ConversationBufferWindowMemory()
    conversation = ConversationChain(
        prompt=PROMPT_TEMPLATE,
        llm=llm,
        memory=memory,
        verbose=False
    )

    return conversation

def generate_code(conversation, prompt):
    print("Model response:")
    output = conversation.invoke(prompt)
    display_markdown(output['response'], raw=True)

In [7]:
# Respuesta del Modelo LLM
conversation = start_conversation()
generate_code(conversation, """Escribir un programa en Python,una función para calcular el interes de un plazo fijo.  Capital inicial, años e interes enviar como parametros.""")


Model response:


Notas:

La función `calcular_interes` calcula el interés simple ganado en un plazo fijo.  Asume un interés anual simple y no considera intereses compuestos.  Los parámetros son:

* `capital_inicial`: El monto inicial invertido.
* `años`: El número de años que dura el plazo fijo.
* `interes`: La tasa de interés anual (expresada como decimal, ej. 5% = 0.05).

La función retorna el interés ganado.  Se incluye manejo de errores para entradas inválidas.


```python
import sys

def calcular_interes(capital_inicial, años, interes):
    """
    Calcula el interés simple ganado en un plazo fijo.

    Args:
        capital_inicial: El monto inicial invertido (debe ser un número positivo).
        años: El número de años que dura el plazo fijo (debe ser un número positivo).
        interes: La tasa de interés anual (debe ser un número entre 0 y 1).

    Returns:
        El interés ganado.  Retorna None si hay un error en los parámetros de entrada.
    """
    # Validación de entradas
    if not all(isinstance(arg, (int, float)) and arg > 0 for arg in [capital_inicial, años, interes]):
        print("Error: Todos los parámetros deben ser números positivos.", file=sys.stderr)
        return None
    if not 0 <= interes <= 1:
        print("Error: La tasa de interés debe estar entre 0 y 1.", file=sys.stderr)
        return None

    # Cálculo del interés simple
    interes_ganado = capital_inicial * años * interes
    return interes_ganado


# Ejemplo de uso
capital = 1000
tiempo = 3
tasa = 0.05

interes_total = calcular_interes(capital, tiempo, tasa)

if interes_total is not None:
    print(f"El interés ganado en {tiempo} años con un capital de ${capital} y una tasa del {tasa*100:.2f}% es: ${interes_total:.2f}")

```

Notas sobre la ejecución:

El código primero define la función `calcular_interes` con validación de entrada para asegurar que los argumentos sean numéricos y positivos, y que la tasa de interés esté dentro del rango válido. Luego, proporciona un ejemplo de cómo usar la función, mostrando el interés ganado después de 3 años con un capital de 1000 y una tasa de interés del 5%.  Si hay algún error en los parámetros de entrada, se imprimirá un mensaje de error en `stderr` y la función retornará `None`.  El resultado se formatea a dos decimales para una mejor legibilidad.

## **Solicitamos un cambio al código generado**



In [8]:
generate_code(conversation, """Podría actualizar el programa para utilizar interes compuesto.""")

Model response:


Notas:

La función `calcular_interes_compuesto` calcula el interés compuesto ganado en un plazo fijo.  Utiliza la fórmula A = P (1 + r/n)^(nt), donde:

* A = el monto final incluyendo interés
* P = capital inicial
* r = tasa de interés anual (como decimal)
* n = número de veces que se capitaliza el interés por año
* t = número de años

La función retorna el interés ganado. Se incluye manejo de errores para entradas inválidas.  Se asume una capitalización anual (n=1) si no se especifica.


```python
import sys

def calcular_interes_compuesto(capital_inicial, años, interes, n=1):
    """
    Calcula el interés compuesto ganado en un plazo fijo.

    Args:
        capital_inicial: El monto inicial invertido (debe ser un número positivo).
        años: El número de años que dura el plazo fijo (debe ser un número positivo).
        interes: La tasa de interés anual (debe ser un número entre 0 y 1).
        n: El número de veces que se capitaliza el interés por año (default 1, anual).

    Returns:
        El interés ganado. Retorna None si hay un error en los parámetros de entrada.
    """
    # Validación de entradas
    if not all(isinstance(arg, (int, float)) and arg > 0 for arg in [capital_inicial, años, interes, n]):
        print("Error: Todos los parámetros deben ser números positivos.", file=sys.stderr)
        return None
    if not 0 <= interes <= 1:
        print("Error: La tasa de interés debe estar entre 0 y 1.", file=sys.stderr)
        return None

    # Cálculo del interés compuesto
    monto_final = capital_inicial * (1 + interes / n)**(n * años)
    interes_ganado = monto_final - capital_inicial
    return interes_ganado


# Ejemplo de uso con interés compuesto
capital = 1000
tiempo = 3
tasa = 0.05

interes_total_compuesto = calcular_interes_compuesto(capital, tiempo, tasa)
interes_total_compuesto_mensual = calcular_interes_compuesto(capital, tiempo, tasa, 12) #Capitalización mensual

if interes_total_compuesto is not None:
    print(f"El interés compuesto ganado en {tiempo} años con un capital de ${capital} y una tasa del {tasa*100:.2f}% (anual) es: ${interes_total_compuesto:.2f}")

if interes_total_compuesto_mensual is not None:
    print(f"El interés compuesto ganado en {tiempo} años con un capital de ${capital} y una tasa del {tasa*100:.2f}% (mensual) es: ${interes_total_compuesto_mensual:.2f}")

```

Notas sobre la ejecución:

El código actualizado define la función `calcular_interes_compuesto`, que calcula el interés compuesto utilizando la fórmula correcta.  Incluye un parámetro opcional `n` para especificar la frecuencia de capitalización (por defecto es anual, n=1).  El ejemplo de uso ahora muestra tanto el interés compuesto anual como mensual.  La validación de entrada sigue siendo la misma, asegurando que los argumentos sean válidos antes de realizar el cálculo.  Los resultados se muestran formateados a dos decimales.

## **Solictamos un segundo cambio al código generado**

In [9]:
generate_code(conversation, """Podría actualizar el programa para que considere aportaciones mensuales.""")

Model response:


Notas:

La función `calcular_interes_compuesto_con_aportaciones` calcula el interés compuesto considerando aportaciones mensuales adicionales.  Utiliza un bucle para iterar sobre cada mes, calculando el interés y añadiendo la aportación mensual. La fórmula básica de interés compuesto se aplica en cada iteración.

```python
import sys

def calcular_interes_compuesto_con_aportaciones(capital_inicial, años, interes, aportacion_mensual, n=12):
    """
    Calcula el interés compuesto con aportaciones mensuales adicionales.

    Args:
        capital_inicial: El monto inicial invertido (debe ser un número positivo).
        años: El número de años que dura el plazo fijo (debe ser un número positivo).
        interes: La tasa de interés anual (debe ser un número entre 0 y 1).
        aportacion_mensual: La aportación mensual (debe ser un número positivo o cero).
        n: El número de veces que se capitaliza el interés por año (default 12, mensual).

    Returns:
        El interés ganado. Retorna None si hay un error en los parámetros de entrada.
    """
    # Validación de entradas
    if not all(isinstance(arg, (int, float)) and arg >= 0 for arg in [capital_inicial, años, interes, aportacion_mensual, n]):
        print("Error: Todos los parámetros deben ser números no negativos.", file=sys.stderr)
        return None
    if not 0 <= interes <= 1:
        print("Error: La tasa de interés debe estar entre 0 y 1.", file=sys.stderr)
        return None

    # Cálculo del interés compuesto con aportaciones mensuales
    meses = años * n
    monto_actual = capital_inicial
    for _ in range(int(meses)):
        monto_actual *= (1 + interes / n)  # Interés ganado
        monto_actual += aportacion_mensual  # Aportación mensual

    interes_ganado = monto_actual - capital_inicial - (aportacion_mensual * meses)
    return interes_ganado


# Ejemplo de uso con interés compuesto y aportaciones mensuales
capital = 1000
tiempo = 3
tasa = 0.05
aportacion = 100

interes_total_compuesto_con_aportaciones = calcular_interes_compuesto_con_aportaciones(capital, tiempo, tasa, aportacion)

if interes_total_compuesto_con_aportaciones is not None:
    print(f"El interés compuesto ganado en {tiempo} años con un capital de ${capital}, una tasa del {tasa*100:.2f}% (mensual) y una aportación mensual de ${aportacion} es: ${interes_total_compuesto_con_aportaciones:.2f}")

```

Notas sobre la ejecución:

El código actualizado define la función `calcular_interes_compuesto_con_aportaciones`, que calcula el interés compuesto considerando aportaciones mensuales.  Se itera mes a mes, aplicando la fórmula de interés compuesto y sumando la aportación mensual.  El ejemplo de uso muestra cómo calcular el interés con una aportación mensual de $100. La validación de entrada se ha actualizado para permitir aportaciones mensuales iguales a cero.  El resultado se muestra formateado a dos decimales.  Se asume una capitalización mensual (n=12) por defecto.

## **Probamos el programa generado por el LLM**

In [ ]:
import sys

def calcular_interes_compuesto_con_aportaciones(capital_inicial, años, interes, aportacion_mensual):
    """
    Calcula el interés compuesto ganado en un plazo fijo con aportaciones mensuales.

    Args:
        capital_inicial: El capital inicial invertido (debe ser un número positivo).
        años: El número de años de la inversión (debe ser un número positivo).
        interes: La tasa de interés anual (debe ser un número positivo).
        aportacion_mensual: La aportación mensual adicional (debe ser un número positivo o cero).

    Returns:
        El interés ganado, o None si los parámetros de entrada son inválidos.
    """
    # Validación de parámetros de entrada
    if not all(isinstance(arg, (int, float)) and arg >= 0 for arg in [capital_inicial, años, interes, aportacion_mensual]):
        print("Error: El capital inicial, los años, la tasa de interés y la aportación mensual deben ser números no negativos.")
        return None

    # Cálculo del interés compuesto con aportaciones mensuales
    meses = int(años * 12)
    capital_actual = capital_inicial
    aportaciones_totales = 0

    for _ in range(meses):
        capital_actual += aportacion_mensual
        aportaciones_totales += aportacion_mensual
        capital_actual *= (1 + interes / (100 * 12)) # Interés mensual

    interes_ganado = capital_actual - capital_inicial - aportaciones_totales
    return interes_ganado


if __name__ == "__main__":
    try:
        capital_inicial = float(input("Ingrese el capital inicial: "))
        años = float(input("Ingrese el número de años: "))
        interes = float(input("Ingrese la tasa de interés anual (%): "))
        aportacion_mensual = float(input("Ingrese la aportación mensual: "))

        interes_total = calcular_interes_compuesto_con_aportaciones(capital_inicial, años, interes, aportacion_mensual)

        if interes_total is not None:
            print(f"El interés ganado después de {años} años es: {interes_total:.2f}")

    except ValueError:
        print("Error: Ingrese valores numéricos válidos.")
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")
        sys.exit(1)

Ingrese el capital inicial: 10000
Ingrese el número de años: 14
Ingrese la tasa de interés anual (%): 8
Ingrese la aportación mensual: 300
El interés ganado después de 14.0 años es: 63157.66
